In [57]:
import numpy as np
import numpy.linalg as lin
import scipy
from sklearn.decomposition import NMF

### Algorithms for NMF
 - Suppose we know $W$. Then finding $H$ becomes (non-negative) matrix least squares:
 
 $ H = \text{argmin}_{V\in\mathbb{R}^{r\times n}_{+}}\|A - WV\|^{2}_{F}$
 
 - What if we knew $H$? Then finding $W$ is also (non-negative) matrix least squares:
 
 $W = \text{argmin}_{U\in\mathbb{R}^{d\times r}_{+}}\|A - UH\|^{2}_{F}$ 
 
 - **Idea:** If we have approximations $W_{k},H_{k}$:
     + Use $W_{k}$ to find new approx. of $H$:
         $ H_{k+1} = \text{argmin}_{V\in\mathbb{R}^{r\times n}_{+}}\|A - W_{k}V\|^{2}_{F} = \text{argmin}_{\mathbf{v}_{i} \geq 0 \ \text{ for } j=1,\ldots n} \sum_{j=1}^{n}\|\mathbf{a}_{j} - W_{k}\mathbf{v}_j\|_{2}^{2}$
     + Note that $V = \left[\begin{matrix} \mathbf{v}_1 & \ldots & \mathbf{v}_{r} \end{matrix}\right]$
     + $\mathbf{v}_{i} = \text{nnls}(W_{k},\mathbf{a}_i)$ for $i=1,\ldots, n$
     + $\|A - W_{k}V\|^{2}_{F} = \sum_{i=1}^{n}\sum_{j=1}^{d}\left(A_{ij} - \left(W_{k}V\right)_{ij}\right)^{2}$
     + Can do some rewriting: $A_{ij} = \mathbf{a}_{i,j}$
     + Then, use $H_{k+1}$ to find a new approx. of $W$:
         $ W_{k+1} = \text{argmin}_{U\in\mathbb{R}^{d\times r}_{+}}\|A - UH_{k+1}\|^{2}_{F}$
     + Extra steps: Know that $\|A - UH_{k+1}\|^{2}_{F} = \|\left(A - UH_{k+1}\right)^{\top}\|^{2}_{F} = \|A^{\top} - H_{k+1}^{\top}U^{\top}\|^{2}_{F}$
     + $\mathbf{u}_i = \text{nnls}(H_{k+1}^{\top},\mathbf{r}_i)$ where $\mathbf{r}_i$ is the $i$-th row of $A$
     + Repeat.
     
     $A\in R^{n\times d}$  you want $W \in R^{n\times r}$ and $H \in R^{r \times d}$ with r $\leq$ min{n,d}. If r is large you can find a good solution in terms of error (i.e. $||A−WH||_{F}$ will be small) but it may lack interpretability.

In [104]:
# Question 1 need to make nonnegative
# X is dim(4,4) => n = 4, d = 4 
A = np.array([[0.238, 0.387, 1.065, 0.494],
             [0.345, 0.603, 1.056, 0.512],
             [0.302, 0.555, 0.59, 0.308],
             [0.283, 0.473, 1.132, 0.531]])

def my_NMF(A):
    # Initalize random W
    W = np.random.rand(4, 2) # n = 4, r = 2
    #H = np.random.rand(2, 4) # r = 2, d = 4
    
    # I got rid of this part below because i was getting errors with trying to 
    # make the function terminate based on the error of W-W_prev || H-H_prev
    # inital error:
    # W_error = 1 
    # H_error = 1
    
    # set tolerance of W and H:
    # tol = 10**-4 
    # W_error <= tol and H_error <= tol
    
    i = 0
    
    while(i < 20):
        H = lin.lstsq(W, A, rcond=None)[0] # fix W and get argmin H of ||A-WH||
        W = lin.lstsq(H.T, A.T, rcond=None)[0].T # repeat but fix H to get W 
        i += 1
        
    return W, H

W,H = my_NMF(A)
print(np.dot(W,H))

[[0.23766663 0.38717405 1.06504525 0.49392641]
 [0.34512935 0.60294615 1.05604023 0.51189323]
 [0.30186981 0.5550608  0.58998738 0.30804221]
 [0.28326086 0.47285479 1.13192647 0.53114686]]


In [101]:
X = np.array([[1, 4],[2, 5]])
Y = np.array([[-2, 2],[3, -3]])
Z = np.array([[1, 0],[0, -1]])

XY = np.multiply.outer(X, Y)
XYZ = np.multiply.outer(XY, Z)

print(XYZ)

[[[[[[ -2   0]
     [  0   2]]

    [[  2   0]
     [  0  -2]]]


   [[[  3   0]
     [  0  -3]]

    [[ -3   0]
     [  0   3]]]]



  [[[[ -8   0]
     [  0   8]]

    [[  8   0]
     [  0  -8]]]


   [[[ 12   0]
     [  0 -12]]

    [[-12   0]
     [  0  12]]]]]




 [[[[[ -4   0]
     [  0   4]]

    [[  4   0]
     [  0  -4]]]


   [[[  6   0]
     [  0  -6]]

    [[ -6   0]
     [  0   6]]]]



  [[[[-10   0]
     [  0  10]]

    [[ 10   0]
     [  0 -10]]]


   [[[ 15   0]
     [  0 -15]]

    [[-15   0]
     [  0  15]]]]]]


In [111]:
X = [-0.82, -1.33, -3.63, -1.62, -2.95, 0.95, 0.53, 1.79, 0.95, 2.28]
mu = 2.37
Y = []

for x in X:
    Y.append(lin.norm(x-mu))
    
#print(Y)

x1, x2 = np.array_split(X,2)
print(x1, x2)
print(np.mean(x1), np.mean(x2))

[-0.82 -1.33 -3.63 -1.62 -2.95] [0.95 0.53 1.79 0.95 2.28]
-2.07 1.3
